In [ ]:
from gensim import corpora, models, utils, parsing
from gensim.test.utils import datapath
from collections import defaultdict, Counter
from pprint import pprint
import os
import re
import string
import pandas as pd
from itertools import islice
import csv
import pandas as pd
import numpy as np

In [ ]:
# this function loads previously saved model 
lda = models.LdaModel.load('lk-ldamodelmallet-new.lda')

In [ ]:
# load topics; for now just load the top 10 words
# note that for tome, we'd need to replace num_words=10 with num_words=10000 <-- !!!!
topics = lda.show_topics(num_topics=100, num_words=10, formatted=False)

# print the first two topic lines just to be sure
for i in range(2):
   print(topics[i]) 

In [ ]:
doc_topics_file = "./topic-data/allpapers-doctopics.txt"

# print the first line
with open(doc_topics_file) as myfile:
    head = [next(myfile) for x in range(1)]
print(head)


In [ ]:
topics_file = "./topic-data/allpapers-topics.txt"

# print the first 2 lines
with open(topics_file) as myfile:
    head = [next(myfile) for x in range(2)]
print(head)

In [ ]:
# now get top 10 words associated with each topic, and print to a file for future reference

cleantopics = ""

for topic in topics:
    wordpairs = topic[1]
    words = []
    wordstring = ""
    
    for wordpair in wordpairs:
        words.append(wordpair[0])
    for word in words:
        wordstring += word + ","
    cleantopics += str(topic[0]) + "," + wordstring + "\n"

with open("./topic-data/20190204-cleantopics-new.csv","w") as myfile:
    myfile.write(cleantopics)     

In [ ]:
# now chunk topic counts by newspaper
# need to end up with csv in format TITLE, TOTALTOPIC0SCORE, TOTALTOPIC1SCORE, TOTALTOPIC2SCORE, ETC.
# the topic scores come from doc_topics_file
# but need to cross reference w/ "aa-docMetadata-new.csv" file to figure out the newspaper that the doc comes from

# first, assign metadata file

metadata_file = "./metadata/aa-docMetadata-new.csv" 

In [ ]:
# first need to figure out number/title of all newspapers

# let's see what the metadata file looks like

# print the first 2 lines
with open(metadata_file) as myfile:
    head = [next(myfile) for x in range(2)]
print(head)



In [ ]:
# so, split by "," and then second item by "/"
metadata = []

with open(metadata_file, newline="\n") as csvfile:
    metadata = list(csv.reader(csvfile))

newspapers = []
    
for doc in metadata:
    docID = doc[0]
    titleString = doc[1]
    titleParts = titleString.split("/")
    title = titleParts[0]
    
    if title not in newspapers:
        newspapers.append(title)
        
print(newspapers)
    

In [ ]:
# now we've got a list of all the newspaper titles, so let's start counting! 
# rememeber, goal is aggregate topic scores for each newspaper 

# create new dataframe w/ newspapers as index, 100 cols, initialized to 0
df = pd.DataFrame(0, index=newspapers, columns=range(100))

df = df.astype('float')

print(df)

In [ ]:
# open the giant doc_topics file and go line by line
with open(doc_topics_file) as myfile:
    for i, line in enumerate(myfile):
        # figure out which newspaper it belongs to; 
        # i is same as docID in metadata file 
        print("Looking up topic values for doc",i)
        doc_metadata = metadata[i]
        titleString = doc_metadata[1]
        titleParts = titleString.split("/")
        title = titleParts[0]
   
        # then, get the topic scores
        items = line.split(",")
        topic_scores = items[1::2]

        # now add to correct topic count for newspaper
    
        for j, score in enumerate(topic_scores):  
            score = score.strip(' ()[]\n' )
            score = float(score)
            df.loc[title].at[j] += score          

In [ ]:
# print to csv

df.to_csv("20190204-newspaper-topics.csv")